In [ ]:
import time

import os
import sys
import tempfile

import numpy as np
import pandas as pd
import openslide

sys.path.insert(0, '../src/python')
from digipath_toolkit import *

os.listdir('../data/run_files')

In [ ]:
# %%writefile ../data/run_files/registration_pairs_to_dir.yml
# method:                 registration_to_dir

# wsi_filename:           ../../DigiPath_MLTK_data/RegistrationDevData/54742d6c5d704efa8f0814456453573a.tiff
# wsi_floatname:          ../../DigiPath_MLTK_data/RegistrationDevData/e39a8d60a56844d695e9579bce8f0335.tiff
# offset_data_file:       ../../DigiPath_MLTK_data/RegistrationDevData/wsi_pair_sample.csv
# output_dir:             ../../run_dir/results

# class_label:            test-label-name
# thumbnail_divisor:      20
# patch_select_method:    threshold_rgb2lab
# rgb2lab_threshold:      80

# image_level:            5
# patch_height:           224
# patch_width:            224
# threshold:              0


# #       New Parameters

# #                       patch_stride_fraction:  1.0 is edge to edge 0.5 is half overlapped (4 times as many patches)
# patch_stride_fraction:  0.5

# #                       These offsets will be filled in from the offset_data_file
# offset_x:               0
# offset_y:               0

# float_offset_x:         0
# float_offset_y:         0


In [ ]:
def registration_pair_to_directory(run_parameters):
    """ Usage: registration_pair_to_directory(run_parameters)
    
    """
    # insert the offset into the run_parmaters:
    if os.path.isfile(run_parameters['offset_data_file']):
        offset_df = pd.read_csv(run_parameters['offset_data_file'])

        offset_x = int(round(offset_df['truth_offset_x'].iloc[0]))
        offset_y = int(round(offset_df['truth_offset_y'].iloc[0]))

        run_parameters['offset_x'] = offset_x
        run_parameters['offset_y'] = offset_y
        run_parameters['float_offset_x'] = -1 * offset_x
        run_parameters['float_offset_y'] = -1 * offset_y

    # form named parameters
    patch_size = (run_parameters['patch_width'], run_parameters['patch_height'])

    float_offset_x = run_parameters['float_offset_x']
    float_offset_y = run_parameters['float_offset_y']

    image_level = run_parameters['image_level']
    #                         ~~~~~~~~~~~~~~~~~~~~~~~<0><0>~~~~~~~~~~~~~~~~~~~~~~~
    for k, v in run_parameters.items():
        print('%25s: %s'%(k,v))
    #                         ~~~~~~~~~~~~~~~~~~~~~~~<0><0>~~~~~~~~~~~~~~~~~~~~~~~
    image_file_name = run_parameters['wsi_filename']

    if os.path.isfile(image_file_name) == False:
        print('\n\n\tFile not found:\n\t%s\n\n'%(image_file_name))
        # return

    output_dir = run_parameters['output_dir']
    class_label = run_parameters['class_label']

    if 'file_ext' in run_parameters:
        file_ext = run_parameters['file_ext']
        if file_ext[0] != '.':
            file_ext = '.' + file_ext
    else:
        file_ext = '.jpg'

    # check / ceate the output directory
    if os.path.isdir(output_dir) == False:
        print('creating output directory:\n%s\n' % (output_dir))
        os.makedirs(output_dir)

    # prepare name generation dictionary
    _, file_name_base = os.path.split(image_file_name)
    file_name_base, _ = os.path.splitext(file_name_base)

    # sanitize case_id, class_label & file_ext so that they may be decoded - warns user if parameter changes
    file_name_base, class_label = patch_name_parts_clean_with_warning(file_name_base, class_label)

    patch_image_name_dict = {'case_id': file_name_base, 'class_label': class_label, 'file_ext': file_ext}


    # get the patch image generator
    fixed_im_generator = PatchImageGenerator(run_parameters)

    # Open the float image 
    wsi_float_obj = openslide.OpenSlide(run_parameters['wsi_floatname'])
    X, Y = wsi_float_obj.dimensions
    X_dim = (0, X)
    Y_dim = (0, Y)

    image_level = run_parameters['image_level']
    test_count = 0
    stop_teration = False
    patch_dict = fixed_im_generator.next_patch()
    while stop_teration == False:
        # form the patch filename
        patch_image_name_dict['location_x'] = patch_dict['image_level_x']
        patch_image_name_dict['location_y'] = patch_dict['image_level_y']
        patch_name = dict_to_patch_name(patch_image_name_dict)
        patch_full_name = os.path.join(output_dir, patch_name)

        # get the full-scale image location of the patch
        x1 = patch_dict['level_0_x'] + float_offset_x 
        y1 = patch_dict['level_0_y'] + float_offset_y 

        # define the patch full boundry
        x_b = [x1, x1 + patch_size[0]]
        y_b = [y1, y1 + patch_size[1]]

        # check bounds, get float image, write the pair
        if check_patch_in_bounds(x_b, y_b, X_dim, Y_dim) == True:
            flot_im = wsi_float_obj.read_region((x1, y1), image_level, patch_size)
            im_pair = im_pair_hori(patch_dict['patch_image'], flot_im)

            # write the file
            im_pair.convert('RGB').save(patch_full_name)

        try:
            patch_dict = fixed_im_generator.next_patch()
        except:
            print('Stop Iteration')
            stop_teration = True
            pass
        #                         ~~~~~~~~~~~~~~~~~~~~~~~<0><0>~~~~~~~~~~~~~~~~~~~~~~~
        test_count += 1
        if test_count > 5:
            stop_teration = True
        #                         ~~~~~~~~~~~~~~~~~~~~~~~<0><0>~~~~~~~~~~~~~~~~~~~~~~~    

    # clean up and quit
    try:
        del fixed_im_generator
        print('deleted: fixed_im_generator')
        wsi_float_obj.close()
        print('wsi_float_obj: closed')
    except:
        print('already deleted')
        pass

In [ ]:
test_dir = '../../D_run_dir'
if os.path.isdir(test_dir) == False:
    os.makedirs(test_dir)
    
run_parameters = get_run_parameters(run_directory='../data/run_files', run_file='registration_pairs_to_dir.yml')

test_yaml_file = os.path.join(run_parameters['run_directory'], run_parameters['run_file'])
if os.path.isfile(test_yaml_file):
    try:
        os.system('cp ' + test_yaml_file + ' ' + test_dir)
    except:
        print('Copy Fails')
        pass
    
run_parameters['run_directory'] = '../../D_run_dir'

for k, v in run_parameters.items():
    print('%25s: %s'%(k,v))
    
registration_pair_to_directory(run_parameters)